<a href="https://colab.research.google.com/github/shernee/06_CMPE297/blob/main/Mistral_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install optimum
!pip install auto-gptq
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.

In [ ]:
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.llms import CTransformers

An example of using this in production would be to create a vector store of all the documents of the organization. (Below is an example of creating an embedding of just one document due to compute constraints). Once the embeddings are created using Mistral and stored in chromadb, we can create a QnA retrieval chain using LangChain to retrieve relevant and grounded answers.

In [ ]:
mna_news = """JSW Energy on August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore.
Sajjan Jindal led JSW group had signed an exclusivity agreement with Hyderabad based Mytrah Energy to buy the latter’s wind and solar assets.
This is the largest acquisition by JSW Energy since inception, which comprises 17 special purpose vehicles and 1 ancillary SPV.  The transaction is subject to approval of the Competition Commission of India (CCI) and other customary approvals standard to a transaction of this size, the company said in a release.
With this acquisition, JSW Energy's current  operational generation capacity will go up by over 35 percent from 4,784 MW to 6,537 MW.  Currently about 2,500 MW of under-construction wind and hydro projects which are likely to be commissioned in phases over the next 18-24 months, JSW Energy platform capacity gets enhanced to 9.1 GW – where the share of renewables increases to  65 percent, JSW said in a stock exchange filing. Further, this is likely to help the company in achieving its renewable-led capacity growth target of 10 GW by FY25, well ahead of timelines, the firm added.
"""

documents = [Document(page_content=mna_news, metadata={"source": "local"})]

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_file = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
config = {'max_new_tokens': 256, 'temperature': 0.5}

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=model_name)#, model_file=model_file)

KeyboardInterrupt: ignored

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
retriever = vectordb.as_retriever()

In [ ]:
llm = CTransformers(model=model_name, model_file=model_file, config=config)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
def run_my_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

query =""" Which is the buyer and the seller company here? """
run_my_rag(qa, query)